<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Biaxial-compression-of-a-rectangular-box-filled-with-disks" data-toc-modified-id="Biaxial-compression-of-a-rectangular-box-filled-with-disks-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Biaxial compression of a rectangular box filled with disks</a></span><ul class="toc-item"><li><span><a href="#Pre-processing" data-toc-modified-id="Pre-processing-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Pre-processing</a></span></li></ul></li><li><span><a href="#Computation" data-toc-modified-id="Computation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Computation</a></span></li></ul></div>

# Biaxial compression of a rectangular box filled with disks

F. Dubois - 2016

## Pre-processing

Initialisations

In [ ]:
from pathlib import Path
import math
import numpy as np

from pylmgc90 import pre

datbox = Path('DATBOX')
datbox.mkdir(exist_ok=True)

# 2D
dim = 2

Defining empty containers

In [ ]:
# containers
#   * bodies
bodies = pre.avatars()
#   * models
mods = pre.models()
#   * materials
mats = pre.materials()
#   * see tables
svs  = pre.see_tables()
#   * contact laws
tacts = pre.tact_behavs()

Defining model and material

In [ ]:
# creation of a rigid model
mod = pre.model(name='rigid', physics='MECAx', element='Rxx2D', dimension=dim)
mods.addModel(mod)
# creation of two materials
tdur = pre.material(name='TDURx',materialType='RIGID',density=1000.)
plex = pre.material(name='PLEXx',materialType='RIGID',density=100.)
mats.addMaterial(tdur,plex)

Generating particles

In [ ]:
# 1000 particles generation
nb_particles=1000

# random distribution in [0.5, 2.[ 
radii=pre.granulo_Random(nb_particles, 0.5, 2.)

# minimum and maximum radii
radius_min= np.amin(radii)
radius_max= np.amax(radii)

Putting it in a box 

In [ ]:
# deposit in a rectangular box
lx = 75.
ly = 50. 
nb_remaining_particles, coor, radii = pre.depositInBox2D(radii, lx, ly)

# check if all particles were deposited
if (nb_remaining_particles < nb_particles):
    print("Warning: granulometry changed, since some particles were removed!")

# disk addition loop:
for r, c in zip(radii, coor):
    # creation of a new rigid disk made of 'plex' material
    body=pre.rigidDisk(r=r, center=c, model=mod, material=plex, color='BLUEx') 
    # add the disk to the avatar container
    bodies += body

Adding some "walls" to the box

In [ ]:
# add a smooth box, i.e made of JONCx:

# declaring one body per wall
down = pre.rigidJonc(axe1=0.5*lx+radius_max, axe2=radius_max, center=[0.5*lx, -radius_max],
                     model=mod, material=tdur, color='WALLx')
up   = pre.rigidJonc(axe1=0.5*lx+radius_max, axe2=radius_max, center=[0.5*lx, ly+radius_max],
                     model=mod, material=tdur, color='WALLx')
left = pre.rigidJonc(axe1=0.5*ly+radius_max, axe2=radius_max, center=[-radius_max, 0.5*ly],
                     model=mod, material=tdur, color='WALLx')
right= pre.rigidJonc(axe1=0.5*ly+radius_max, axe2=radius_max, center=[lx+radius_max, 0.5*ly],
                     model=mod, material=tdur, color='WALLx')

# rotating the vertical walls (with respect to the its inertia center)
left.rotate(psi=-math.pi/2., center=left.nodes[1].coor)
right.rotate(psi=math.pi/2., center=right.nodes[1].coor)

# fix walls
down.imposeDrivenDof(component=[1, 2, 3], dofty='vlocy')
up.imposeDrivenDof(component=[1, 3], dofty='vlocy')
left.imposeDrivenDof(component=[1, 2, 3], dofty='vlocy')
right.imposeDrivenDof(component=[2, 3], dofty='vlocy')

# add the walls to the avatar container
bodies += down; bodies += up; bodies += left; bodies += right

Applying loads

In [ ]:
# predefined : [ ct + amp * cos(omega*t+phi) ] * sgn(rampi + ramp*t) * min(abs(rampi + ramp*t), 1)

up.imposeDrivenDof(component=2, dofty='force',description='predefined',
                   ct=10.0, amp=0.0, omega=0.0, phi=0.0, rampi=1.0, ramp=0.0)

# evolution : giving a file containing t,f(t)

right.imposeDrivenDof(component=1,dofty='vlocy',description='evolution',evolutionFile='vx.dat')

# more details:
# help(up.imposeDrivenDof)

Building an evolution file:

In [ ]:
t0=0.5
t1 =1.
vx =0.1

def imposedVx(t):
    # 0 until t0
    if t <= t0:
        return 0.
    # linear growing between [t0, t1]
    elif t > t0 and t <= t1:
        return -vx*(t-t0)/(t1-t0)
    # constant value afterward
    else:
        return -vx

pre.writeEvolution(f=imposedVx, instants=np.linspace(0., 2*t1, 1000) ,path='DATBOX/', name='vx.dat')

Defining contact laws and see tables

In [ ]:
# interaction management:
#   * law declaration
#       - between particles
ldkdk = pre.tact_behav(name='iqsc0',law='IQS_CLB',fric=0.3)
tacts+= ldkdk
#       - with walls
ldkjc = pre.tact_behav(name='iqsc1',law='IQS_CLB',fric=0.5)
tacts+= ldkjc

# see possible tact_behav
print(pre.config.lmgc90dicts.tactBehavOptions.keys())
# parameters of 'VEL_SGR_CLB'
print(pre.config.lmgc90dicts.tactBehavOptions['VEL_SGR_CLB'])

Defining see tables

In [ ]:
#   * visibility table declaration
#       - between particles
svdkdk = pre.see_table(CorpsCandidat   ='RBDY2', candidat   ='DISKx', colorCandidat   ='BLUEx', behav=ldkdk,
                       CorpsAntagoniste='RBDY2', antagoniste='DISKx', colorAntagoniste='BLUEx',
                       alert=0.1*radius_min)
svs+=svdkdk
#       - with walls
svdkjc = pre.see_table(CorpsCandidat   ='RBDY2', candidat   ='DISKx', colorCandidat   ='BLUEx', behav=ldkjc,
                       CorpsAntagoniste='RBDY2', antagoniste='JONCx', colorAntagoniste='WALLx',
                       alert=0.1*radius_min)
svs+=svdkjc

Writting to files

In [ ]:
# writting files
pre.writeDatbox(dim, mats, mods, bodies, tacts, svs, datbox_path='DATBOX')

Displaying

In [ ]:
#try:
#    pre.visuAvatars(bodies)
#except:
#    pass

Postprocessing

In [ ]:
post=pre.postpro_commands()
post.addCommand(pre.postpro_command(name='SOLVER INFORMATIONS', step=1))
post.addCommand(pre.postpro_command(name='VIOLATION EVOLUTION', step=1))
pre.writePostpro(commands=post, parts=bodies, path='DATBOX/')

# possible commands
print(pre.config.lmgc90dicts.commandOptions.keys())
# parameters of 'BODY TRACKING'
print(pre.config.lmgc90dicts.commandOptions['BODY TRACKING'])

# Computation

To run a the corresponding computation, just to check that the model is alright, the following cells can be run:

In [ ]:
dim = 2

# time evolution parameters
dt = 1e-3
nb_steps = 500

# theta integrator parameter
theta = 0.5

# nlgs parameters
tol    = 1e-4
relax  = 1.0
norm   = 'Quad '
gs_it1 = 500
gs_it2 = 10
stype  = 'Stored_Delassus_Loops         '

# write parameter
freq_write   = 10
#freq_write   =  5

# display parameters
freq_display = 10

In [ ]:
from pylmgc90.chipy import computation
help(computation.initialize)
help(computation.one_step)

In [ ]:
computation.initialize(dim, dt, theta)
#computation.initialize(dim, dt, theta, restart=50)
#computation.initialize(dim, dt, theta, restart=(100,51))
for k in range( 1, nb_steps+1 ):
    if k%50 == 0:
        print( f"computing step {k}" )
    computation.one_step(stype, norm, tol ,relax, gs_it1, gs_it2, freq_write, freq_display)
computation.finalize()

In [ ]:
!paraview